In [1]:
# Imports
import json
from collections import OrderedDict
import pprint

## Definitions

### Transitions Definition

In [2]:
# Json input
input_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ "I1 & I2 & !I4" ],
            [ ["I3"], ["I4"] ]
        ],
        ["P3"],
        "T1 comment"
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ "I4 & !I3" ]
        ],
        ["P1", "P2"],
        "T2 comment"
    ]
}
"""

In [3]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [4]:
# Expected NuSMV
expected= "STABLE:=0;\n" + \
        "WHILE STABLE=0 DO\n" + \
        "   STABLE:=1;\n" + \
        "   (*T1*)\n" + \
        "   IF (P1 AND P2) AND ( (I1 AND I2 AND NOT I4) OR (I3 AND NOT I4) ) AND (NOT P3) THEN\n" + \
        "       P1:=0;\n" + \
        "       P2:=0;\n" + \
        "       P3:=1;\n" + \
        "       STABLE:=0;\n" + \
        "   END_IF;\n" + \
        "   (*T2*)\n" + \
        "   IF (P3) AND ( (I4 AND NOT I1 AND NOT I2) OR (I4 AND NOT I3) ) AND (NOT P1 AND NOT P2) THEN\n" + \
        "       P3:=0;\n" + \
        "       P1:=1;\n" + \
        "       P2:=1;\n" + \
        "       STABLE:=0;\n" + \
        "   END_IF;\n" + \
        "END_WHILE;\n"

In [5]:
out_str= "STABLE:=0;\n" + \
    "WHILE STABLE=0 DO\n" + \
    "   STABLE:=1;\n"

In [6]:
for key in iter(in_dict):

    # Get value of current transition
    value= in_dict[key]

    # Pre-places
    pre= value[0]
    # Pre-places condition
    pre_str= "("
    # Pre-places assignment
    pre_ass_str= ""
    for idx,item in enumerate(pre):
        # Not last element
        if idx < len(pre)-1:
            pre_str= pre_str + item + " AND "
        else:
            pre_str= pre_str + item
        # Assignment
        pre_ass_str= pre_ass_str + "       " + item + ":=0;\n"

    pre_str= pre_str + ")"

    # Inputs
    req_inputs= value[1]
    # Required inputs condition
    req_str= "( "
    for idx_out,item in enumerate(req_inputs):

        # Temp string opening brace
        temp_str= "("

        # Check if condition is specified as a raw string
        if isinstance(item[0], str):
            # Replace & with AND
            temp_temp= item[0].replace("&", "AND")
            # Replace | with OR
            temp_temp= temp_temp.replace("|", "OR")
            # Replace ! with NOT
            temp_temp= temp_temp.replace("!", "NOT ")

            temp_str= temp_str + temp_temp
        
        else:
            high_ins= item[0]
            low_ins= item[1]
            # temp inputs
            for idx_high,high_in in enumerate(high_ins):
                # Not last item
                if idx_high < len(high_ins)-1:
                    temp_str= temp_str + high_in + " AND "
                # If last item
                else:
                    # If there are no low outputs required
                    if len(low_ins) < 1:
                        temp_str= temp_str + high_in
                    else:
                        temp_str= temp_str + high_in + " AND "
            
            # Low inputs
            for idx_low,low_in in enumerate(low_ins):
                # Not last item
                if idx_low < len(low_ins)-1:
                    temp_str= temp_str + "NOT " + low_in + " AND "
                # If last item
                else:
                    temp_str= temp_str + "NOT " + low_in

        # Temp string closing brace
        temp_str= temp_str + ")"   

        # If not last item, add an OR operator
        if idx_out < (len(req_inputs)-1):
            temp_str= temp_str + " OR "

        # Append Temp string to req_str
        req_str= req_str + temp_str

    # req_str closin brace
    req_str= req_str + " )"

    # Post-places
    post= value[2]
    # Post-places condition
    post_str= "("
    # Post places assignment
    post_ass_str= ""
    for idx_post,item_post in enumerate(post):
        # Not last element
        if idx_post < len(post)-1:
            post_str= post_str + "NOT " + item_post + " AND "
        else:
            post_str= post_str + "NOT " + item_post
        # Assignment
        post_ass_str= post_ass_str + "       " + item_post + ":=1;\n"

    post_str= post_str + ")"

    # Append transition name to output
    out_str= out_str + "   (*" + key + "*)\n"
    # Append if statement
    out_str= out_str + "   IF "
    # Append pre-place condition
    out_str= out_str + pre_str + " AND "
    # Append inputs condition
    out_str= out_str + req_str + " AND "
    # Append post-place condition
    out_str= out_str + post_str
    # Add then keyword
    out_str= out_str + " THEN\n"
    # Assignments
    out_str= out_str + pre_ass_str
    out_str= out_str + post_ass_str
    # Stable
    out_str= out_str + "       STABLE:=0;\n"
    # End if
    out_str= out_str + "   END_IF;\n"


In [7]:
out_str= out_str + "END_WHILE;\n"

In [8]:
pprint.pprint(out_str)

('STABLE:=0;\n'
 'WHILE STABLE=0 DO\n'
 '   STABLE:=1;\n'
 '   (*T1*)\n'
 '   IF (P1 AND P2) AND ( (I1 AND I2 AND NOT I4) OR (I3 AND NOT I4) ) AND (NOT '
 'P3) THEN\n'
 '       P1:=0;\n'
 '       P2:=0;\n'
 '       P3:=1;\n'
 '       STABLE:=0;\n'
 '   END_IF;\n'
 '   (*T2*)\n'
 '   IF (P3) AND ( (I4 AND NOT I1 AND NOT I2) OR (I4 AND NOT I3) ) AND (NOT P1 '
 'AND NOT P2) THEN\n'
 '       P3:=0;\n'
 '       P1:=1;\n'
 '       P2:=1;\n'
 '       STABLE:=0;\n'
 '   END_IF;\n'
 'END_WHILE;\n')


In [9]:
pprint.pprint(expected)

('STABLE:=0;\n'
 'WHILE STABLE=0 DO\n'
 '   STABLE:=1;\n'
 '   (*T1*)\n'
 '   IF (P1 AND P2) AND ( (I1 AND I2 AND NOT I4) OR (I3 AND NOT I4) ) AND (NOT '
 'P3) THEN\n'
 '       P1:=0;\n'
 '       P2:=0;\n'
 '       P3:=1;\n'
 '       STABLE:=0;\n'
 '   END_IF;\n'
 '   (*T2*)\n'
 '   IF (P3) AND ( (I4 AND NOT I1 AND NOT I2) OR (I4 AND NOT I3) ) AND (NOT P1 '
 'AND NOT P2) THEN\n'
 '       P3:=0;\n'
 '       P1:=1;\n'
 '       P2:=1;\n'
 '       STABLE:=0;\n'
 '   END_IF;\n'
 'END_WHILE;\n')


In [10]:
assert(out_str == expected)

### Output Definition

In [11]:
# Json input
input_str= """
{
    "PS2": [
        [
            ["RP_AL1_ST_CLAMP"], ["AL1_ST_GRAB"]
        ],
        "Product gets withing graber range"
    ],
    "PS4": [
        [
            ["RP_AL1_ST_CLAMP"], ["AL1_ST_GRAB"]
        ],
        ["AL1_ST_Z_SET:=750"],
        "Move grabber down to pick up product"
    ],
    "PSE0": [
        [
            ["AL1_ST_GRAB"], ["AL1_EMIT", "RC_AL1_ST"]
        ],
        "New box arrived, stop conveyor"
    ],
    "initial": ["PS2", "PSE0"]
    
}
"""

In [12]:
# Expected output
expected= "(*AL1_ST_GRAB*)\n" + \
            "IF PSE0 THEN\n" + \
            "   AL1_ST_GRAB:=1;\n" + \
            "END_IF;\n" + \
            "(*RP_AL1_ST_CLAMP*)\n" + \
            "IF PS2 OR PS4 THEN\n" + \
            "   RP_AL1_ST_CLAMP:=1;\n" + \
            "END_IF;\n" + \
            "(*AL1_EMIT*)\n" + \
            "IF PSE0 THEN\n" + \
            "   AL1_EMIT:=0;\n" + \
            "END_IF;\n" + \
            "(*AL1_ST_GRAB*)\n" + \
            "IF PS2 OR PS4 THEN\n" + \
            "   AL1_ST_GRAB:=0;\n" + \
            "END_IF;\n" + \
            "(*RC_AL1_ST*)\n" + \
            "IF PSE0 THEN\n" + \
            "   RC_AL1_ST:=0;\n" + \
            "END_IF;\n" + \
            "(*AL1_ST_Z_SET*)\n" + \
            "IF PS4 THEN\n" + \
            "   AL1_ST_Z_SET:=750;\n" + \
            "END_IF;\n" 

In [13]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)


In [14]:
in_dict

OrderedDict([('PS2',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               'Product gets withing graber range']),
             ('PS4',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               ['AL1_ST_Z_SET:=750'],
               'Move grabber down to pick up product']),
             ('PSE0',
              [[['AL1_ST_GRAB'], ['AL1_EMIT', 'RC_AL1_ST']],
               'New box arrived, stop conveyor']),
             ('initial', ['PS2', 'PSE0'])])

In [15]:
# Get set of boolean outputs
output_set= set()

for key in list(in_dict):
    if not key == "initial":
        for output in in_dict[key][0][0]:
            output_set.add(output)
        for output in in_dict[key][0][1]:
            output_set.add(output)


In [16]:
# Sort alphabetically
output_set= sorted(output_set)

In [17]:
print(output_set)

['AL1_EMIT', 'AL1_ST_GRAB', 'RC_AL1_ST', 'RP_AL1_ST_CLAMP']


In [18]:
# Collate list of places for which each output is set/reset
output_dict= OrderedDict()

for output in output_set:
    # Initialize list of places for which output is set/reset
    output_dict[output]= [[],[]]

    for key in list(in_dict):
        # Add places for which output is set
        if output in in_dict[key][0][0]:
            output_dict[output][0].append(key)

        # Add places for which output is reset
        if output in in_dict[key][0][1]:
            output_dict[output][1].append(key)


In [19]:
print(output_dict)

OrderedDict([('AL1_EMIT', [[], ['PSE0']]), ('AL1_ST_GRAB', [['PSE0'], ['PS2', 'PS4']]), ('RC_AL1_ST', [[], ['PSE0']]), ('RP_AL1_ST_CLAMP', [['PS2', 'PS4'], []])])


In [20]:
# Generate output string
set_output_str= ""
reset_output_str= ""

# Loop through output keys
for key in list(output_dict):
    output= output_dict[key]

    # If the output is set at at least one place
    if len(output[0]) > 0:
        # Get set places
        set_output_str= set_output_str + "(*" +   key + "*)\n"
        set_output_str= set_output_str + "IF "
        for idx_set,place_set in enumerate(output[0]):
            # Not last item
            if idx_set < len(output[0])-1:
                set_output_str= set_output_str + output[0][idx_set] + " OR "
            # Last Item
            else:
                set_output_str= set_output_str + output[0][idx_set] + " THEN\n"
            
        set_output_str= set_output_str + "   " + key + ":=1;\n" + "END_IF;\n"

    
    # If output is reset at at least 1 place
    if len(output[1]) > 0:
        # Get reset places
        reset_output_str= reset_output_str + "(*" +   key + "*)\n"
        reset_output_str= reset_output_str + "IF " 
        for idx_reset,place_reset in enumerate(output[1]):
            # Not last item
            if idx_reset < len(output[1])-1:
                reset_output_str= reset_output_str + output[1][idx_reset] + " OR "
            # Last Item
            else:
                reset_output_str= reset_output_str + output[1][idx_reset] + " THEN\n"
            
        reset_output_str= reset_output_str + "   " + key + ":=0;\n" + "END_IF;\n"



### Non-boolean outputs

In [21]:
# Non-boolean outputs
# Get dictionary of outputs, the places that set them, and their assigned values
output_dict= dict()

for key in list(in_dict):
    if not key == "initial":
        # Check if exists
        if in_dict[key][1] and isinstance(in_dict[key][1], list):
            # Get string
            val= in_dict[key][1][0]
            # Split string on :=
            output_name, aa, output_val= val.partition(":=") 
            output_dict[output_name]= [key, output_val]


In [22]:
output_dict

{'AL1_ST_Z_SET': ['PS4', '750']}

In [23]:
nb_output_str= ""

In [24]:
# Compose string to set outputs
for key in list(output_dict):
    nb_output_str= nb_output_str + "(*" + key + "*)\n" + \
        "IF " + output_dict[key][0] + " THEN\n" + \
        "   " + key + ":=" + output_dict[key][1] + ";\n" + \
        "END_IF;\n"

In [25]:
nb_output_str

'(*AL1_ST_Z_SET*)\nIF PS4 THEN\n   AL1_ST_Z_SET:=750;\nEND_IF;\n'

In [26]:
out_str= set_output_str + reset_output_str  + nb_output_str

In [27]:
pprint.pprint(out_str)

('(*AL1_ST_GRAB*)\n'
 'IF PSE0 THEN\n'
 '   AL1_ST_GRAB:=1;\n'
 'END_IF;\n'
 '(*RP_AL1_ST_CLAMP*)\n'
 'IF PS2 OR PS4 THEN\n'
 '   RP_AL1_ST_CLAMP:=1;\n'
 'END_IF;\n'
 '(*AL1_EMIT*)\n'
 'IF PSE0 THEN\n'
 '   AL1_EMIT:=0;\n'
 'END_IF;\n'
 '(*AL1_ST_GRAB*)\n'
 'IF PS2 OR PS4 THEN\n'
 '   AL1_ST_GRAB:=0;\n'
 'END_IF;\n'
 '(*RC_AL1_ST*)\n'
 'IF PSE0 THEN\n'
 '   RC_AL1_ST:=0;\n'
 'END_IF;\n'
 '(*AL1_ST_Z_SET*)\n'
 'IF PS4 THEN\n'
 '   AL1_ST_Z_SET:=750;\n'
 'END_IF;\n')


In [28]:
pprint.pprint(expected)

('(*AL1_ST_GRAB*)\n'
 'IF PSE0 THEN\n'
 '   AL1_ST_GRAB:=1;\n'
 'END_IF;\n'
 '(*RP_AL1_ST_CLAMP*)\n'
 'IF PS2 OR PS4 THEN\n'
 '   RP_AL1_ST_CLAMP:=1;\n'
 'END_IF;\n'
 '(*AL1_EMIT*)\n'
 'IF PSE0 THEN\n'
 '   AL1_EMIT:=0;\n'
 'END_IF;\n'
 '(*AL1_ST_GRAB*)\n'
 'IF PS2 OR PS4 THEN\n'
 '   AL1_ST_GRAB:=0;\n'
 'END_IF;\n'
 '(*RC_AL1_ST*)\n'
 'IF PSE0 THEN\n'
 '   RC_AL1_ST:=0;\n'
 'END_IF;\n'
 '(*AL1_ST_Z_SET*)\n'
 'IF PS4 THEN\n'
 '   AL1_ST_Z_SET:=750;\n'
 'END_IF;\n')


In [29]:
assert (out_str == expected)

### Internal variable Definition

In [2]:
#Json input
input_str= r"""
{
    "timer_P1": [
        [
            "MODULE",
            {
                "Q": "boolean",
                "ET": [
                    "{\"zero\", \"half\", \"full\"}",
                    "zero",
                    "{\"zero\", \"half\", \"full\"}"
                ]
            }
        ],
        [
            "TON",
            "timer_P1(IN:= P1, PT:= T#100ms)"
        ]
    ],
    "count": [
        [
            "0..10",
            0,
            "{0, 3, 7, 10}"
        ],
        [
            "INT",
            "count:=0"
        ]
    ],
    "control1": [
        [
            "boolean",
            "{TRUE, FALSE}",
            "{TRUE, FALSE}"
        ],
        [
            "BOOL",
            "control1:=0"
        ]
    ]
}
"""

In [3]:
# Expected NuSMV
expected= "timer_P1(IN:= P1, PT:= T#100ms);\n" +\
    "count:=0;\n" +\
    "control1:=0;\n" 

In [4]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [5]:
in_dict

OrderedDict([('timer_P1',
              [['MODULE',
                OrderedDict([('Q', 'boolean'),
                             ('ET',
                              ['{"zero", "half", "full"}',
                               'zero',
                               '{"zero", "half", "full"}'])])],
               ['TON', 'timer_P1(IN:= P1, PT:= T#100ms)']]),
             ('count', [['0..10', 0, '{0, 3, 7, 10}'], ['INT', 'count:=0']]),
             ('control1',
              [['boolean', '{TRUE, FALSE}', '{TRUE, FALSE}'],
               ['BOOL', 'control1:=0']])])

In [11]:
out_str= ""

In [12]:
try:
    # Loop through elements
    for key in list(in_dict):
        out_str += "{0};\n".format(in_dict[key][1][1])
except:
    pass


In [13]:
out_str

'timer_P1(IN:= P1, PT:= T#100ms);\ncount:=0;\ncontrol1:=0;\n'

In [14]:
assert expected == out_str